In [1]:
import os
import time
from dotenv import load_dotenv
import speech_recognition as sr
from datasets import load_dataset
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import pyttsx3  # For Text-to-Speech functionality

# Initialize environment
load_dotenv()
recognizer = sr.Recognizer()

/Users/amjad/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# 1. Knowledge Base Initialization
def initialize_knowledge_base():
    """Create or load FAISS vector store with Wikipedia data"""
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    
    if not os.path.exists("faiss.index"):
        print("Creating new knowledge base...")
        dataset = load_dataset("wikipedia", "20220301.simple", split="train[:1000]", trust_remote_code=True)
        
        # Process Wikipedia articles
        documents = [
            Document(
                page_content=article["text"],
                metadata={"title": article["title"], "source": "Wikipedia Simple"}
            ) for article in dataset
        ]
        
        # Split and index documents
        splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=150)
        chunks = splitter.split_documents(documents)
        vectorstore = FAISS.from_documents(chunks, embeddings)
        vectorstore.save_local("faiss.index")
        print("Knowledge base created!")
    
    # Load existing index
    return FAISS.load_local("faiss.index", embeddings, allow_dangerous_deserialization=True)


In [3]:
# 2. Travel Assistant Prompt Template
TRAVEL_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""You are WanderGuide, an expert travel assistant. Respond in a friendly, engaging tone with emojis where appropriate.

1. Context: {context}
2. The user is asking: {question}

Respond by:
- Providing a **clear and friendly summary** of the city or itinerary information (20 sentences).
- Highlighting **Top 3 attractions** (include entry prices and timings).
- Suggesting **local specialties** (food, drinks, etc.).
- Offering **transportation tips** (e.g., public transit, taxis).
- Creating **detailed itineraries** (morning, afternoon, and evening activities with time estimates).
- Mentioning any **safety or cultural tips**.
- **Formatting:** Use bullet points, bold key words, and avoid excessive detail beyond 1000 words.

Question: {question}

WanderGuide:"""
)

In [4]:
# 3. Audio Processing Functions
def listen_to_user(timeout=10):
    """Convert speech to text with error handling"""
    try:
        with sr.Microphone() as source:
            print("\n🎤 Listening... (Speak now)")
            audio = recognizer.listen(source, timeout=timeout)
        return recognizer.recognize_google(audio)
    except sr.WaitTimeoutError:
        print("⌛ No speech detected, please try again!")
        return ""
    except Exception as e:
        print(f"🔇 : {str(e)}. Try speaking again.")
        return ""

def text_to_speech(text):
    """Convert text to speech"""
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)  # Speed of speech
    engine.setProperty('volume', 1)  # Volume level (0.0 to 1.0)
    engine.say(text)
    engine.runAndWait()

def get_input_method():
    """Get user's preferred input method"""
    while True:
        method = input("\nChoose input method [text/audio]: ").lower()
        if method in ["text", "t"]:
            return input("✍️ Your travel question: ")
        elif method in ["audio", "a"]:
            return listen_to_user()
        print("⚠️ Please enter 'text' or 'audio'")


In [5]:
# 4. Memory Integration using LangChain
# Initialize memory to hold conversation context
memory = ConversationBufferMemory()

/var/folders/9s/snfcxp3j4gv4r29jbzrzlgsr0000gn/T/ipykernel_53116/2215258845.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [6]:
# 5. Response Processing
def format_advice(response):
    """Format the assistant's response with emphasis on clarity and readability"""
    result = f"\n🌍 HERE's YOUR TRAVEL GUIDE:\n\n{response['result']}\n"

    # Add sources if available
    sources = {doc.metadata.get('title') for doc in response['source_documents']}
    if sources:
        result += "\n📚 Information sources:\n" + "\n".join(f"- **{s}**" for s in sources if s)

    return result

def process_text_query(query):
    """Process text input with memory"""
    memory.append({"input": query, "output": ""})  # Store the input in memory
    response = memory.append_and_predict(input=query)  # Process the query with memory
    return response


In [ ]:
# 6. Main Application
def main():
    """Run travel assistant session"""
    # Initialize components
    vectorstore = initialize_knowledge_base()
    qa_chain = RetrievalQA.from_chain_type(
        llm=OpenAI(temperature=0.3, model_name="gpt-3.5-turbo-instruct"),
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
        chain_type_kwargs={"prompt": TRAVEL_PROMPT},
        return_source_documents=True
    )

    # Welcome message
    print("\n" + "="*55)
    print("🌟 Welcome to WanderGuide - Your Smart Travel Assistant 🌟")
    print("="*55)
    print("\nAsk me about:\n- Cities\n- Travel plans\n- Cultural tips\n- Historical sites")

    # Interaction loop
    while True:
        try:
            # Get user query (text or audio)
            query = get_input_method()  # Get the input once
            if not query.strip():
                continue

            # Process query
            start_time = time.time()
            if any(kw in query.lower() for kw in ["plan", "itinerary", "schedule"]):
                response = qa_chain.invoke({"query": f"Create detailed travel plan: {query}"})
            else:
                response = qa_chain.invoke({"query": query})

            # Show response and use TTS for spoken feedback
            formatted_response = format_advice(response)
            print(formatted_response)
            text_to_speech(formatted_response)

            # Print memory buffer to track conversation history
            print(f"\n💭 Current Memory: {memory.buffer}")

            print(f"\n⏱️  Response time: {time.time()-start_time:.1f}s")

        except KeyboardInterrupt:
            print("\n🛫 Happy travels! Closing WanderGuide...")
            break

if __name__ == "__main__":
    main()


🌟 Welcome to WanderGuide - Your Smart Travel Assistant 🌟

Ask me about:
- Cities
- Travel plans
- Cultural tips
- Historical sites

🎤 Listening... (Speak now)

🌍 HERE's YOUR TRAVEL GUIDE:

 🇸🇦 Welcome to Saudi Arabia! 🌴 As you explore this beautiful country, you'll find a diverse range of local foods influenced by Middle Eastern, African, and Indian cuisines. 🍽️ Some must-try dishes include:

- Kabsa: a rice and meat dish seasoned with spices like cardamom, saffron, and cinnamon. 🍚
- Shawarma: thinly sliced meat (usually chicken or lamb) wrapped in flatbread with vegetables and sauces. 🌯
- Falafel: fried balls made from ground chickpeas and spices, often served in a pita with vegetables and tahini sauce. 🧆

For a taste of traditional Saudi Arabian breakfast, try:

- Ful medames: a hearty dish made from fava beans, served with bread and toppings like eggs, cheese, and vegetables. 🍳
- Haneeth: slow-roasted lamb or goat served with rice, bread, and a spicy tomato sauce. 🍖
- Balaleet: a s